# <u>Métodos No Supervisados - EXTRA

# Comparación de Cluster solo Usando la métrica de Silueta

## Librerias

In [ ]:
#!pip install pyclustering

In [ ]:
from pyclustering.cluster.clarans import clarans;
from pyclustering.utils import timedcall;
from pyclustering.cluster.kmedoids import kmedoids;
import pandas as pd
import numpy as np

# visualization
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
basket = pd.read_csv("data/basketball.csv")
basket.head(8)

### Analizamos la data

In [ ]:
basket.info()

In [ ]:
sns.pairplot(basket)

## Clustering Jerarquico

### Normalizamos los datos

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit(basket)
normalized = scaler.transform(basket)
normalized1 = pd.DataFrame(normalized,columns=list(basket.columns))
zscore_df = normalized1
zscore_df.head(10)

### Estandarizamos los datos

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler = scaler.fit(basket)
standarizar = scaler.transform(basket)
standarizar1 = pd.DataFrame(standarizar,columns=list(basket.columns))
zscore_df1 = standarizar1
zscore_df1.head(10)

## Dendograma

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage  
from matplotlib import pyplot as plt

seed = 16
np.random.seed(seed)

linked = linkage(zscore_df,method = 'ward')

labelList = range(1, 11)

plt.figure(figsize=(16, 8))  
dendrogram(linked)
plt.show()  

## Construimos el Modelo Jerarquico con 4 cluster

In [ ]:
from sklearn.cluster import AgglomerativeClustering

seed = 16
np.random.seed(seed)

cluster_Ag = AgglomerativeClustering(n_clusters=4, affinity='euclidean', linkage='ward')  
cluster_Ag.fit_predict(zscore_df)   ## Recordar que aca se reemplaza su dataframe

In [ ]:
label_ag = pd.DataFrame(cluster_Ag.labels_,columns= ['Cluster'])
cluster_Jerarq = pd.concat([basket,label_ag['Cluster']], axis=1)
cluster_Jerarq

In [ ]:
cluster_Ag.labels_

In [ ]:
cluster_Jerarq.columns

## Visualizacion en 2D

In [ ]:
plt.figure(figsize=(10, 7))  
plt.scatter(zscore_df['asisten_porcen_minuto'], zscore_df['tiempo_juego'], c=cluster_Ag.labels_ , cmap='rainbow') 

## Visualizacion en 3D

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

plt.rcParams['figure.figsize'] = (18, 9)
fig = plt.figure()
ax = Axes3D(fig)
ax.scatter(zscore_df.asisten_porcen_minuto, zscore_df.tiempo_juego, zscore_df.edad, c=cluster_Ag.labels_ ,cmap='rainbow')

In [ ]:
from sklearn.metrics import silhouette_score
silhouette_score(basket.values, cluster_Ag.labels_, metric='euclidean')

## Usamos Kmedoides con P.A.M

Debemos de fijar la cantidad de cluster

In [ ]:
seed = 16 # con esto definimos la semilla o con random_state
np.random.seed(seed)

kmedoids_instance = kmedoids(zscore_df.values, [3, 6, 4,10],random_state=16); ## para probar mas cluster solo agregar en
# la lista [3, 6, 4] aca seria para 3 cluster si quieres 4 agrega [3, 6, 4, 10] un valor cualesquiera por que son aleatorios

#calls the clarans method 'process' to implement the algortihm
(ticks, result) = timedcall(kmedoids_instance.process);
print("Execution time : ", ticks, "\n");

#returns the clusters 
clusters_Kmed = kmedoids_instance.get_clusters();

#returns the mediods 
medoids = kmedoids_instance.get_medoids();

print("Index of the points that are in a cluster : ",clusters_Kmed)
print("The index of medoids that algorithm found to be best :", medoids)

Adecuamos los cluster con etiquetas

In [ ]:
label = pd.DataFrame()
for i in range(4):  # Esto es para 3 cluster si quiero 4 cambio aca el valor anda mas  
    print(i)
    a = pd.DataFrame(clusters_Kmed[i],columns = ['Index']) 
    a['Cluster'] = i
    label = label.append(a)

In [ ]:
label

In [ ]:
label_f = label.sort_values(by ='Index').reset_index(drop='true')
cluster_Med = pd.concat([basket,label_f['Cluster']], axis=1)
cluster_Med

In [ ]:
cluster_Med.Cluster.value_counts()

## Visualizacion en 2D

In [ ]:
plt.figure(figsize=(10, 7))  
plt.scatter(zscore_df['asisten_porcen_minuto'], zscore_df['puntos_porcen_minuto'], c=cluster_Med.Cluster, cmap='rainbow') 

## Visualizacion en 3D

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

plt.rcParams['figure.figsize'] = (18, 9)
fig = plt.figure()
ax = Axes3D(fig)
ax.scatter(zscore_df.asisten_porcen_minuto, zscore_df.tiempo_juego, zscore_df.edad, c=cluster_Med.Cluster,cmap='rainbow')

### Métrica de Silueta

In [ ]:
from sklearn.metrics import silhouette_score
silhouette_score(basket.values, cluster_Med.Cluster, metric='euclidean')

## Usamos Kmeans

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=4,random_state=16)
kmeans.fit(zscore_df.values)
centroids = kmeans.cluster_centers_
print("The index of kemans that algorithm found to be best :", kmeans.labels_)

Adecuamos los cluster con etiquetas

In [ ]:
label_km = pd.DataFrame(kmeans.labels_,columns= ['Cluster'])
cluster_kmeans = pd.concat([basket,label_km['Cluster']], axis=1)
cluster_kmeans

In [ ]:
cluster_kmeans.Cluster.value_counts()

## Visualizacion en 2D

In [ ]:
plt.figure(figsize=(10, 7))  
plt.scatter(zscore_df['asisten_porcen_minuto'], zscore_df['puntos_porcen_minuto'], c=cluster_kmeans.Cluster, cmap='rainbow') 

## Visualizacion en 3D

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

plt.rcParams['figure.figsize'] = (18, 9)
fig = plt.figure()
ax = Axes3D(fig)
ax.scatter(zscore_df.asisten_porcen_minuto, zscore_df.puntos_porcen_minuto, zscore_df.edad, c=cluster_kmeans.Cluster ,cmap='rainbow')

### Métrica de Silueta

In [ ]:
from sklearn.metrics import silhouette_score
silhouette_score(basket.values, cluster_kmeans.Cluster, metric='euclidean')